In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
data = pd .read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [0]:
geography = pd.get_dummies(data['Geography'], drop_first=True)
gender = pd.get_dummies(data['Gender'], drop_first=True)

In [0]:
X = data.iloc[:, 3:13]
y = data.iloc[:, 13]

In [0]:
X = pd.concat([X, geography, gender], axis=1)

X = X.drop(['Geography', 'Gender'], axis=1)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [0]:
#Hyperparameter Optimization

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.activations import relu, sigmoid

from keras.layers import Dropout

In [0]:
def creat_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0 :
            model.add(Dense(nodes, input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else :
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
    model.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [0]:
model = KerasClassifier(build_fn=creat_model, verbose=0)

In [0]:
layers = [[20], [20, 15], [20, 30, 15]]
activations = ['relu', 'sigmoid']
pram_grid = dict(layers=layers, activation=activations, batch_size=[128,256], epochs=[30])

In [0]:
grid = GridSearchCV(estimator=model, param_grid=pram_grid, cv=5)

grid_search = grid.fit(X_train, y_train)

In [19]:
[grid_search.best_score_,grid_search.best_params_]

[0.8475,
 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [20]}]